## Prepping (joining) datasets for DB

In [17]:
import pandas as pd
import os

samples_check = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/samples_check.csv")
morpho = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/morpho.csv")
lab_analysis = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/analysis.csv")
site_info = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/site_info.csv")
soil_type = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/soil_type.csv")

profile_record = pd.read_csv("/Users/inesschwartz/Desktop/cleandata/profile_record_clean.csv")

In [1]:
import pandas as pd
import os

# Set the path to your folder containing the 10 CSV files
csv_folder = "/Users/inesschwartz/Desktop/cleandata"  

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Loop through each file and display column data types
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    print(f"\n📄 File: {file}")
    
    try:
        df = pd.read_csv(file_path)
        print(df.dtypes)
    except Exception as e:
        print(f"⚠️ Error reading {file}: {e}")


📄 File: analysis.csv
lab_sample_id      int64
analysis_id       object
horizon_id        object
sample_id         object
profile           object
                  ...   
Tl               float64
Pb               float64
Bi               float64
Th               float64
U                float64
Length: 71, dtype: object

📄 File: soil_type.csv
soil_type_id     int64
profile         object
CEP_GR          object
CEP_NAME        object
FAO             object
dtype: object

📄 File: geology_mapping.csv
geology_code           object
geology_description    object
dtype: object

📄 File: lithology1954_mapping.csv
lithology_1954_code           object
lithology_1954_description    object
dtype: object

📄 File: samples_check.csv
sample_id              int64
site_info_id          object
profile_record_id    float64
profile               object
horizon_id           float64
shelf                 object
room                  object
year                 float64
dtype: object

📄 File: morpho.csv
horizo

In [4]:
# Folder containing the CSVs
csv_folder = "/Users/inesschwartz/Desktop/cleandata"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith(".csv")]

# Define identifier columns
identifier_columns = [
    'sample_id', 'site_info_id', 'profile',
    'horizon_id', 'lab_sample_id', 'climate_id',
    'geo_features_id', 'topo_features_id', 'profile_record_id'
]

# Function to convert ID columns to string type
def convert_identifiers_to_string(df, id_columns):
    """
    Converts specified identifier columns in a DataFrame to string type,
    safely handling float64 values and preserving missing values (NA).
    """
    df = df.copy()
    for col in id_columns:
        if col in df.columns:
            df.loc[:, col] = df[col].apply(
                lambda x: str(int(x)) if pd.notna(x) and isinstance(x, float) and x.is_integer()
                else str(x) if pd.notna(x)
                else pd.NA
            ).astype("string")
    return df

# Loop through and clean all CSV files
for file in csv_files:
    file_path = os.path.join(csv_folder, file)
    df = pd.read_csv(file_path)
    df_cleaned = convert_identifiers_to_string(df, identifier_columns)
    df_cleaned.to_csv(file_path, index=False)
    print(f"✅ Cleaned: {file}")

print("🎯 All identifier columns standardized to string type.")


✅ Cleaned: analysis.csv
✅ Cleaned: soil_type.csv
✅ Cleaned: geology_mapping.csv
✅ Cleaned: lithology1954_mapping.csv
✅ Cleaned: samples_check.csv
✅ Cleaned: morpho.csv
✅ Cleaned: site_info.csv
✅ Cleaned: profile_record_clean.csv
✅ Cleaned: lithology_mapping.csv
✅ Cleaned: topo_feat.csv
✅ Cleaned: geo_feat.csv
✅ Cleaned: climate_feat.csv
🎯 All identifier columns standardized to string type.


An overview of how much data we had to clean out :(

In [12]:
samples_check.shape

(14715, 8)

In [11]:
morpho.shape

(10434, 30)

In [10]:
lab_analysis.shape

(7763, 71)

In [14]:
site_info.shape

(4321, 12)

## completing profile_record_clean table